# B. Sim Dashboard

### Imports

In [1]:
%run "C:\Users\james\Documents\MLB\Code\U1. Imports.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U2. Utilities.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"

In [2]:
# time.sleep(20*60)

In [3]:
# # Schedule
# pause_code(start_time='2024-04-04T15:00:00', timezone='EST')

In [4]:
%run "A. Night Dashboard.ipynb"

Running Night Dashboard
CPU times: total: 0 ns
Wall time: 2min 51s
CPU times: total: 15.6 ms
Wall time: 57.9 s
CPU times: total: 0 ns
Wall time: 3.2 s
Could not scrape RotoGrinders weather data.
19BBD
19BBA
19BBE
19BC0
19BC2
19BC4
19BC7
19BC8
19BC9
Scraping successful!
160862337
160862330
160862338
160862335
160862336
160862061
Draftables 160862061.csv not found.
160862060
Draftables 160862060.csv not found.
160862057
Draftables 160862057.csv not found.
160862059
Draftables 160862059.csv not found.
160862058
Draftables 160862058.csv not found.
160862320
Draftables 160862320.csv not found.
160862316
Draftables 160862316.csv not found.
160861757
Draftables 160861757.csv not found.
160862318
Draftables 160862318.csv not found.
160862321
Draftables 160862321.csv not found.
160862282
160862285
160862280
160862300
160862284
160862121
Draftables 160862121.csv not found.
160862120
Draftables 160862120.csv not found.
160862119
Draftables 160862119.csv not found.
160862118
Draftables 160862118.c

### Directory

In [5]:
baseball_path = r'C:\Users\james\Documents\MLB\Database'

### Contest

In [6]:
# Read in subset of contests for which we'll create guides
subset_df = pd.read_csv(os.path.join(baseball_path, 'A01. DraftKings', '7. Subsets', f'Subset {todaysdate}.csv'))
four_seamer_df = subset_df[subset_df['Name'].str.contains('eamer')].sort_values('contestTime').reset_index(drop=True)
four_seamer_df

,Name,Cash Prize,Entry Fee,contestKey,draftGroupId,contestDate,contestTime,date,date_dash
0,MLB $5K Four-Seamer (Early),"$5,000",4.0,160862208,105402,2024-04-24 13:15:00,Wed 1:15PM,20240424,NaN
1,MLB $1.5K Four-Seamer (Turbo),1500.0,4.0,160862265,105410,2024-04-24 18:40:00,Wed 6:40PM,20240424,2024-04-24
2,MLB $20K Four-Seamer [20 Entry Max],20000.0,4.0,160862284,105412,2024-04-24 19:05:00,Wed 7:05PM,20240424,2024-04-24
3,MLB $1K Four-Seamer (Night),1000.0,4.0,160862338,105416,2024-04-24 20:05:00,Wed 8:05PM,20240424,2024-04-24


In [7]:
# Set contestKey
contestKey = four_seamer_df['contestKey'][2]

# Read in Contest Guide
guide = pd.read_csv(os.path.join(baseball_path, "A09. Contest Guides", f"Contest Guide {contestKey}.csv"))

# Identify draftGroupId
draftGroupId = guide['draftGroupId'][0]

# Identify RotoWire slate
roto_slate = guide['roto_slate'][0]

# Print out name
print(f"Date: {guide['date'][0]}. Slate: {guide['slate'][0]}. {guide['name'][0]}. {roto_slate}.")

Date: 20240424. Slate: All. MLB $20K Four-Seamer [20 Entry Max]. 14993.


### Settings

In [8]:
# Recreate game_df
generate = True

# Run historic matchups
historic = False
# Run matchups necessary for slate
slate_only = True

# Run:
Matchups = True
Simulations = True
Optimizer = True

# Games

Select start and end date (should be todaysdate for both if running day-of)

In [9]:
# start_date = "20220924"
start_date = todaysdate
end_date = todaysdate

Read in games

In [10]:
game_df = read_and_save_games(team_map, generate)
game_df = game_df[(game_df['date'] >= start_date) & (game_df['date'] <= end_date)].reset_index(drop=True)

In [11]:
slate_df = game_df[game_df['game_id'].isin(guide['game_id'].unique())].reset_index(drop=True)

### Weather

In [12]:
rotogrinders_df = pd.read_csv(os.path.join(baseball_path, "A06. Weather", "2. RotoGrinders", f"RotoGrinders {todaysdate}.csv"))
red_list = list(rotogrinders_df.query('Tag == "red" or Tag2 == "red"')['Away']) + list(rotogrinders_df.query('Tag == "red" or Tag2 == "red"')['Home'])
orange_list = list(rotogrinders_df.query('Tag == "orange" or Tag2 == "orange"')['Away']) + list(rotogrinders_df.query('Tag == "orange" or Tag2 == "orange"')['Home'])
rotogrinders_df[['Tag', 'Tag2', 'Away', 'Home', 'date', 'Description']]

,Tag,Tag2,Away,Home,date,Description
0,NaN,NaN,NaN,NaN,20240424,NaN
1,green,NaN,OAK,NYY,20240424,"Winds are blowing out to right in New York today, a slight bump to the bats."


### B01. Matchups

In [13]:
%run "C:\Users\james\Documents\MLB\Code\B01. Matchups.ipynb"
%run "C:\Users\james\Documents\MLB\Code\A02. MLB API.ipynb"
%run "C:\Users\james\Documents\MLB\Code\A03. Steamer.ipynb"

##### Load
This loads in datasets needed to create matchup files

In [14]:
%%time
complete_dataset = pd.read_csv(os.path.join(baseball_path, "Stat Dataset.csv"))
steamer_hitters_df = pd.read_csv(os.path.join(baseball_path, "Steamer Hitters.csv"))
steamer_pitchers_df = pd.read_csv(os.path.join(baseball_path, "Steamer Pitchers.csv"))

CPU times: total: 13.6 s
Wall time: 13.9 s


Create matchups <br>
Why does this sometimes have pickling errors? Fixed by rerunning.

In [15]:
%%time
try:
    print(len(game_df))
    empty_list = Parallel(n_jobs=-1, verbose=True)(delayed(create_matchup_files)(game_df, row, complete_dataset, steamer_hitters_df, steamer_pitchers_df, team_map) for row in range(len(game_df)))
except:
    print("Trying Again")
    empty_list = Parallel(n_jobs=-1, verbose=True)(delayed(create_matchup_files)(game_df, row, complete_dataset, steamer_hitters_df, steamer_pitchers_df, team_map) for row in range(len(game_df)))

15


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.0min finished


CPU times: total: 34.2 s
Wall time: 1min 3s


### B02. Simulations

In [41]:
%run "C:\Users\james\Documents\MLB\Code\B02. Simulations.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U1. Imports.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"

In [17]:
historic, slate_only

(False, True)

In [18]:
num_batches = int(os.cpu_count() / 1)
batch_size = int(63 * 1)

In [19]:
if historic == False:
    # Openers
    draftables = pd.read_csv(os.path.join(baseball_path, "A01. DraftKings", "2. Draftables", f"Draftables {draftGroupId}.csv"), encoding='iso-8859-1')
    opener_list = list(draftables.query('Salary < 5000 and `Roster Position` == "P"')['Name'])
    print(f"Openers: {opener_list[:10]}")

    # Teams with Projected Lineups
    projected_lineups = pd.read_csv(os.path.join(baseball_path, "A05. Rosters", "3. Batting Orders Projected", f"Batting Orders Projected {todaysdate}.csv"))
    projected_teams = projected_lineups.merge(team_map[['SFBBTEAM', 'DKTEAM']], left_on=['team code'], right_on=['SFBBTEAM'], how='inner')
    projected_team_list = list(projected_teams.query('` confirmed` == "N"')['DKTEAM'].unique())
    print(f"Projected teams: {projected_team_list}")
else:
    opener_list = []
    projected_lineups = None
    projected_team_list = []

Openers: ['Noah Murdock', 'Grant Holman', 'Dan Altavilla', 'Eric Pardinho', 'Jesús Tinoco', 'Carson Coleman', 'Daniel Camarena', 'Chance Adams', 'Matt Canterino', 'Edwin Escobar']
Projected teams: []


In [20]:
# Choose which games to run simulations for
if slate_only == True:
    # Just the slate
    run_df = slate_df.copy()
else:
    # All games
    run_df = game_df.copy()

In [21]:
%%time
# Print out games to simulate
_ = [print(f"{row['away_team']}@{row['home_team']} {pd.to_datetime(row['game_datetime']).tz_convert('US/Eastern').strftime('%Y-%m-%d %H:%M:%S')}") for index, row in run_df.head(15).iterrows()]

# Loop over dates of interest
for date in list(run_df['date'].unique()):
    print(date)
    # Extract year
    year = date[:4]
    # Extract matchups
    matchup_list = [os.path.splitext(f)[0] for f in os.listdir(os.path.join(baseball_path, "B01. Matchups", f"Matchups {date}"))]
    
    # Loop over games
    daily_df = run_df.query(f'date == "{date}"').reset_index()
    print(daily_df.shape)
    for i in range(len(daily_df)):            
        # Extract info from run_df to look up proper file in matchups path
        away_team = daily_df['away_team'][i]
        home_team = daily_df['home_team'][i]
        game_id = daily_df['game_id'][i]
        # Beginning of filename in matchup folder
        lookup = f"{away_team}@{home_team} {game_id}"
        
        # Find the matchup file
        matchup = next((matchup for matchup in matchup_list if matchup.startswith(lookup)), None)
        print(matchup)
        
        # Create matchups
        matchup_path = os.path.join(baseball_path, "B01. Matchups", f"Matchups {date}", f"{matchup}.xlsx")
        AwayBatters, HomeBatters, AwayPitchers, HomePitchers = create_matchup(matchup_path, batter_stats_scaler, batter_stats_fg_scaler, batter_imputations_model, pitcher_stats_scaler, pitcher_stats_fg_scaler, pitcher_imputations_model, projected_lineups)
        
        # Create Scoreboard object
        game_template = Scoreboard(AwayBatters, HomeBatters, AwayPitchers, HomePitchers, 9)       
        
        ### Sim games
        # With batching (fast!)
        start = time.time()
        game_list = Parallel(n_jobs=num_batches, verbose=1)(delayed(sim_game_batch)(game_template, model_pulls, model_binary, model_outs, model_safe, opener_list, year, innings=9, debug=False, batch_size=batch_size) for batches in range(num_batches))
        game_list = [game for sublist in game_list for game in sublist]
        print(f"Simming {batch_size*num_batches} games took {time.time() - start} seconds.")
        
        # Without batching (slow!)
        # game_list = Parallel(n_jobs=16, verbose=0)(delayed(sim_game)(game_template, model_pulls, model_binary, model_outs, model_safe, opener_list, year, innings=9, debug=False) for sims in range(num_sims))
        
        # # Without joblib (slower!)
        # start = time.time()
        # game_list = []
        # for i in range(10):
        #     game = sim_game(game_template, model_pulls, model_binary, model_outs, model_safe, opener_list, year, innings=9, debug=True)
        #     game_list.append(game)
        # print(f"Simming {num_sims} games took {time.time() - start} seconds.")
        
        # Create object path
        game_path = os.path.join(baseball_path, "B02. Simulations", "1. Game Sims", f"Matchups {date}", f"{matchup}") 
        player_path = os.path.join(baseball_path, "B02. Simulations", "2. Player Sims", f"Matchups {date}", f"{matchup}") 
        # Clear folders
        if os.path.exists(game_path):
            shutil.rmtree(game_path)
        if os.path.exists(player_path):
            shutil.rmtree(player_path)
        # Make folders
        os.makedirs(game_path, exist_ok=True)
        os.makedirs(player_path, exist_ok=True)
        
        # Save each object in the list to a separate file
        for i, game_object in enumerate(game_list):
            # Delete unnecessary attributes from batter objects
            for batter in game_object.away_batters + game_object.home_batters:
                batter = batter.keep_selected_attributes()
            # Delete unnecessary attributes from pitcher objects
            for pitcher in game_object.away_pitchers + game_object.home_pitchers:
                pitcher = pitcher.keep_selected_attributes()
            # Delete unnecessary attributes from game objects
            game_object.keep_selected_attributes()

            # Constructing DataFrames for game, batters, and pitchers
            game_score_df = pd.DataFrame({
                "away_score": [game_object.away_score],
                "home_score": [game_object.home_score]
            })

            away_batters_df = pd.DataFrame([vars(batter) for batter in game_object.away_batters])
            away_batters_df['team'] = "away"
            home_batters_df = pd.DataFrame([vars(batter) for batter in game_object.home_batters])
            home_batters_df['team'] = "home"
            batters_df = pd.concat([away_batters_df, home_batters_df], axis=0)
            
            away_pitchers_df = pd.DataFrame([vars(pitcher) for pitcher in game_object.away_pitchers])
            away_pitchers_df['team'] = "away"
            home_pitchers_df = pd.DataFrame([vars(pitcher) for pitcher in game_object.home_pitchers])
            home_pitchers_df['team'] = "home"
            pitchers_df = pd.concat([away_pitchers_df, home_pitchers_df], axis=0)
            
            # Specify the file paths where you want to save the CSV files
            game_file_path = os.path.join(game_path, "game_{}.csv").format(i)
            batters_file_path = os.path.join(player_path, "batters_{}.csv").format(i)
            pitchers_file_path = os.path.join(player_path, "pitchers_{}.csv").format(i)

            # Saving DataFrames to CSV files
            game_score_df.to_csv(game_file_path, index=False)
            batters_df.to_csv(batters_file_path, index=False)
            pitchers_df.to_csv(pitchers_file_path, index=False)

        print(f'Saved to {game_path}')

OAK@NYY 2024-04-24 19:05:00
MIA@ATL 2024-04-24 19:20:00
HOU@CHC 2024-04-24 19:40:00
TOR@KCR 2024-04-24 19:40:00
CHW@MIN 2024-04-24 19:40:00
SEA@TEX 2024-04-24 20:05:00
SDP@COL 2024-04-24 20:40:00
20240424
(7, 33)
OAK@NYY 745753 1905
45.0
No pitcher imputations vs RHB
No pitcher imputations vs LHB


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:  1.4min remaining: 10.1min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:  1.6min finished


Simming 1008 games took 96.27312207221985 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240424\OAK@NYY 745753 1905
MIA@ATL 747132 1920
45.0
No pitcher imputations vs RHB
No pitcher imputations vs RHB
No pitcher imputations vs LHB


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:  1.5min remaining: 10.7min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:  1.7min finished


Simming 1008 games took 102.66493654251099 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240424\MIA@ATL 747132 1920
HOU@CHC 746889 1940
45.0
No pitcher imputations vs RHB


...

Simming 1008 games took 96.03044843673706 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240424\CHW@MIN 745919 1940
SEA@TEX 745028 2005
45.0
No pitcher imputations vs RHB
No pitcher imputations vs LHB


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:  1.5min remaining: 10.6min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:  1.7min finished


Simming 1008 games took 101.26291012763977 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240424\SEA@TEX 745028 2005
SDP@COL 746567 2040
45.0


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:  1.4min remaining:  9.9min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:  1.6min finished


Simming 1008 games took 93.19786071777344 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240424\SDP@COL 746567 2040
CPU times: total: 2min 10s
Wall time: 12min 30s


In [22]:
# Optionally, read in C02, vegas scores, my scores

In [23]:
# 20220924
# SHOULD DELETE EXISTING FOLDERS

### B03. Lineups

In [24]:
%run "U1. Imports.ipynb"
%run "U2. Utilities.ipynb"
%run "C02. Projections.ipynb"
%run "B03. Optimizer.ipynb"

In [25]:
sort_by = 'Plus3'
min_salary = 49000
min_projection = (5, 5, 4)
major_stack = (5, 5, 5)
minor_stack = (2, 2, 2)
max_exposure_batters = 0.5
max_exposure_pitchers = 0.7
excluded_teams = ([team for var in ['red_list', 'orange_list', 'projected_team_list'] if globals().get(var) is not None for team in globals()[var]], [team for var in ['red_list'] if globals().get(var) is not None for team in globals()[var]], [team for var in ['red_list'] if globals().get(var) is not None for team in globals()[var]])
min_starters = (10, 10, 0)
lineups = 200
historic= False

### Run One

##### 1. Players

In [26]:
draftables_with_sims = create_player_file(contestKey, guide, draftGroupId, roto_slate, max_exposure_pitchers, max_exposure_batters)
draftables_with_sims.to_csv(os.path.join(baseball_path, "B03. Lineups", "1. Players", f"Players {contestKey}.csv"), index=False, encoding='iso-8859-1')

In [27]:
webbrowser.open(os.path.join(baseball_path, "B03. Lineups", "1. Players", f"Players {contestKey}.csv"))

True

##### 2. Lineups

In [28]:
%%time
attempts = [
    ("Maximum Constraints", min_projection[0], major_stack[0], minor_stack[0], excluded_teams[0], min_starters[0]),
    ("Some Constraints", min_projection[1], major_stack[1], minor_stack[1], excluded_teams[1], min_starters[1]),
    ("Minimum Constraints", min_projection[2], major_stack[2], minor_stack[2], excluded_teams[2], min_starters[2])
]

for attempt_name, projection, major, minor, excluded, starters in attempts:
    print(f"Attempting: {attempt_name}")
    try:
        create_lineups(contestKey, min_salary, projection, major, minor, excluded, starters, lineups)
        break  # Exit loop if successful
    except Exception as e:
        print(f"Attempt failed: {e}\n")

Attempting: Maximum Constraints
1
25
50
75
99
100
125
150
175
CPU times: total: 3.14 s
Wall time: 7min


##### 3. Lineups Ranked

In [29]:
sort_by_list = ['P50', 'P75', 'P90', 'P95', 'P99', 'P100', 'Tail', 'Sim STD', 'Plus2', 'Plus3', 'rostership', 'pitcher rostership', 'batter_rostership']
lineups_ranked = choose_lineups(contestKey, roto_slate, sort_by)
lineups_ranked.to_csv(os.path.join(baseball_path, "B03. Lineups", "3. Lineups Ranked", f"Lineups Ranked {contestKey}.csv"), index=False)

(200, 1100)


In [30]:
lineups_ranked.query('pareto == 1')

P                        P.1  \
0   Jameson Taillon(33783507)         Joe Ryan(33783500)   
1   Jameson Taillon(33783507)         Jon Gray(33783505)   
2          Joe Ryan(33783500)         Jon Gray(33783505)   
3          Joe Ryan(33783500)         Jon Gray(33783505)   
4    Reynaldo Lopez(33783502)  Jameson Taillon(33783507)   
5   Jameson Taillon(33783507)         Joe Ryan(33783500)   
6   Jameson Taillon(33783507)         Joe Ryan(33783500)   
7   Jameson Taillon(33783507)         Joe Ryan(33783500)   
8          Joe Ryan(33783500)         Jon Gray(33783505)   
9   Jameson Taillon(33783507)         Joe Ryan(33783500)   
10  Jameson Taillon(33783507)         Joe Ryan(33783500)   
11  Jameson Taillon(33783507)         Joe Ryan(33783500)   
12  Jameson Taillon(33783507)         Jon Gray(33783505)   
13   Reynaldo Lopez(33783502)  Jameson Taillon(33783507)   
14  Jameson Taillon(33783507)         Joe Ryan(33783500)   
15   Reynaldo Lopez(33783502)         Joe Ryan(33783500)   
16  Jameson Taillon(33783507)         Joe Ryan(33783500)   
17  Jameson Taillon(33783507)         Joe Ryan(33783500)   
18   Reynaldo Lopez(33783502)         Jon Gray(33783505)   
19  Jameson Taillon(33783507)         Jon Gray(33783505)   
20  Jameson Taillon(33783507)         Joe Ryan(33783500)   
21  Jameson Taillon(33783507)         Joe Ryan(33783500)   
22   Reynaldo Lopez(33783502)         Joe Ryan(33783500)   
23  Jameson Taillon(33783507)         Joe Ryan(33783500)   

                            C                          1B  \
0   Kyle Higashioka(33783608)  Elehuris Montero(33783581)   
1   Kyle Higashioka(33783608)  Elehuris Montero(33783581)   
2   Kyle Higashioka(33783608)    Carlos Santana(33783629)   
3   Kyle Higashioka(33783608)  Elehuris Montero(33783581)   
4   Kyle Higashioka(33783608)  Elehuris Montero(33783581)   
5   Kyle Higashioka(33783608)  Elehuris Montero(33783581)   
6   Kyle Higashioka(33783608)        Luke Raley(33783623)   
7   Kyle Higashioka(33783608)  Elehuris Montero(33783581)   
8   Kyle Higashioka(33783608)    Carlos Santana(33783629)   
9   Kyle Higashioka(33783608)  Elehuris Montero(33783581)   
10  Kyle Higashioka(33783608)  Jake Cronenworth(33783554)   
11  Kyle Higashioka(33783608)  Elehuris Montero(33783581)   
12  Kyle Higashioka(33783608)  Jake Cronenworth(33783554)   
13  Kyle Higashioka(33783608)  Jake Cronenworth(33783554)   
14  Kyle Higashioka(33783608)  Jake Cronenworth(33783554)   
15  Kyle Higashioka(33783608)         Ryan Noda(33783658)   
16  Kyle Higashioka(33783608)  Jake Cronenworth(33783554)   
17  Kyle Higashioka(33783608)  Jake Cronenworth(33783554)   
18       Elias Diaz(33783556)  Elehuris Montero(33783581)   
19       Elias Diaz(33783556)  Elehuris Montero(33783581)   
20       Elias Diaz(33783556)  Elehuris Montero(33783581)   
21       Elias Diaz(33783556)  Elehuris Montero(33783581)   
22  Kyle Higashioka(33783608)  Elehuris Montero(33783581)   
23       Elias Diaz(33783556)  Elehuris Montero(33783581)   

                           2B                        3B  \
0   Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
1   Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
2   Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
3   Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
4   Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
5   Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
6   Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
7   Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
8   Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
9   Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
10  Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
11  Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
12  Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
13  Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
14  Xander Bogaerts(33783531)    Eguy Rosario(33783631)   
15  Xander Bogaerts(33783531)    Eguy Rosario(33783631)  

##### 4. Uploads

In [31]:
upload = create_upload_file(contestKey, sort_by)
upload.to_csv(os.path.join(baseball_path, "B03. Lineups", "4. Uploads", f"Upload {contestKey}.csv"), index=False)

##### 5. Entries

In [39]:
entry = create_entry_file(draftGroupId, contestKey)
entry.to_csv(os.path.join(baseball_path, "B03. Lineups", "5. Entries", f"Entries {draftGroupId}.csv"), index=False, encoding='iso-8859-1')

<IPython.core.display.Javascript object>

##### Email

In [33]:
email_upload_file(draftGroupId, contestKey)

##### Upload

In [40]:
upload_entries()

<IPython.core.display.Javascript object>

### Run All

In [35]:
# Create list of contestKeys with available guides
contestKey_list = os.listdir(os.path.join(baseball_path, "A09. Contest Guides"))
contestKey_list = [int(key.replace(".csv", "").split(" ")[2]) for key in contestKey_list]


# Create list of contestKeys with available results
result_contestKey_list = os.listdir(os.path.join(baseball_path, "A01. DraftKings", "6. Player Results"))
result_contestKey_list = [int(key.replace(".csv", "").split(" ")[2]) for key in result_contestKey_list]

# Find the overlap 
available_contestKey_list = list(set(result_contestKey_list) & set(contestKey_list))

In [36]:
# %%time
# fail_list = Parallel(n_jobs=-1, verbose=1)(delayed(create_contest_lineups2)(contestKey, sort_by, min_salary, min_projection[0], major_stack[0], minor_stack[0], excluded_teams=[], lineups, historic=True) for contestKey in available_contestKey_list)

In [37]:
# %%time
# fail_list2 = Parallel(n_jobs=-1, verbose=1)(delayed(create_contest_lineups2)(contestKey, sort_by='Plus3', min_salary=49000, min_projection=4, major_stack=5, minor_stack=2, max_exposure_batters=0.5, max_exposure_pitchers=0.75, excluded_teams=[], lineups=lineups, historic=True) for contestKey in [contestKey for contestKey in fail_list if contestKey is not None])

In [38]:
# %%time
# fail_list3 = Parallel(n_jobs=-1, verbose=1)(delayed(create_contest_lineups2)(contestKey, sort_by='Plus3', min_salary=49000, min_projection=4, major_stack=4, minor_stack=2, max_exposure_batters=0.5, max_exposure_pitchers=0.75, excluded_teams=[], lineups=lineups, historic=True) for contestKey in [contestKey for contestKey in fail_list2 if contestKey is not None])